<a href="https://colab.research.google.com/github/engineer-nicolas/cs50sql/blob/master/lecture_3_Writing/lecture_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 3 - Writing - CS50 SQL harvard



In this notebook, it will be explored how to add, update, and delete data in databases.

The Boston MFA (Museum of Fine Arts) is a century-old museum in Boston. The MFA manages a vast collection of historical and contemporary artifacts and artwork. We will focus now on the creation (or insertion) of data in a Boston MFA database.

In [1]:
# Let's import the required libraries
import sqlite3
import pandas as pd
import os

In [2]:
#Delete only if it already exists
DB_NAME = "mfa.db"

# Safety check: delete if the filename is exactly what we expect
if os.path.exists(DB_NAME) and DB_NAME.endswith(".db"):
    os.remove(DB_NAME)

Let's create a database called mfa.db.

In [3]:
# Connect to the database file
# it will create the file if it doesn't exist
conn = sqlite3.connect('mfa.db')
# MFA stands for the Boston Museum of Fine Arts (MFA)


In [4]:
conn.execute("""
CREATE TABLE "collections" (
    "id" INTEGER,
    "title" TEXT NOT NULL,
    "accession_number" TEXT NOT NULL UNIQUE,
    "acquired" NUMERIC,
    PRIMARY KEY("id")
);
""")


# Commit changes to save the table in the database
conn.commit()

# Let's inspect the schema of the collections table
pd.read_sql_query(
    "PRAGMA table_info(collections);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,title,TEXT,1,None,0
2,2,accession_number,TEXT,1,None,0
3,3,acquired,NUMERIC,0,None,0


## INSERT INTO ___ VALUES ___


This SQL statement is used to insert a row of data into a given table.



In [5]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("id", "title", "accession_number", "acquired")
VALUES (9, 'Profusion of flowers', '56.257', '1956-04-12');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12


This command requires the list of columns in the table that will receive new data and the values to be added to each column, in the *same order*.

IMPORTANT: SQLite can fill out the primary key values automatically. To make use of this functionality, we omit the ID column altogether while inserting a row.

In [6]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES ('Farmers working at dawn', '11.6152', '1911-08-03');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03


SQLite selects the highest primary key value in the table and increments it to generate the next primary key value — in this case, 10.

### Inserting Multiple Rows

Let us now insert two new paintings into the collections table.

In [7]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Imaginative landscape', '56.496', NULL),
('Peonies and butterfly', '06.1899', '1906-01-01');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03
2,11,Imaginative landscape,56.496,None
3,12,Peonies and butterfly,06.1899,1906-01-01


## IMPORT CSV FILE

SQLite makes it possible to import a CSV file directly into our database. We can import the CSV by running a SQLite command:

`.import --csv --skip 1 mfa.csv collections`

The first argument, --csv indicates to SQLite that we are importing a CSV file.

The second argument indicates that the first row of the CSV file (the header row) needs to be skipped, or not inserted into the table.




## INSERT INTO ___ + SELECT ___ FROM - Inserting from another table

In [8]:
csv_file = """title,accession_number,acquired
Profusion of flowers,56.257,1956-04-12
Farmers working at dawn,11.6152,1911-08-03
Spring outing,14.769,1914-01-08
Red landscape,56.476,
Peonies and butterfly,06.1899,1906-01-01
"""

with open("temp.csv", "w", encoding="utf-8") as f:
    f.write(csv_file)

# Read the CSV file into a DataFrame
df = pd.read_csv("temp.csv")
print(df)

                     title  accession_number    acquired
0     Profusion of flowers           56.2570  1956-04-12
1  Farmers working at dawn           11.6152  1911-08-03
2            Spring outing           14.7690  1914-01-08
3            Red landscape           56.4760         NaN
4    Peonies and butterfly            6.1899  1906-01-01


In [9]:
conn.execute('''
CREATE TABLE IF NOT EXISTS temp (
    title TEXT,
    accession_number TEXT,
    acquired NUMERIC
);
''')
conn.commit()

df.to_sql(
    'temp',
    conn,
    if_exists='replace',
    index=False
)

5

In [10]:
pd.read_sql_query('SELECT * FROM temp;', conn)


,title,accession_number,acquired
0,Profusion of flowers,56.2570,1956-04-12
1,Farmers working at dawn,11.6152,1911-08-03
2,Spring outing,14.7690,1914-01-08
3,Red landscape,56.4760,None
4,Peonies and butterfly,6.1899,1906-01-01


In [11]:
conn.execute("""
INSERT OR IGNORE INTO "collections" ("title", "accession_number", "acquired")
SELECT "title", "accession_number", "acquired"
FROM "temp";
""")
conn.commit()
pd.read_sql_query(
    'SELECT * FROM collections;',
    conn
)


,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03
2,11,Imaginative landscape,56.496,None
3,12,Peonies and butterfly,06.1899,1906-01-01
4,13,Spring outing,14.769,1914-01-08
5,14,Red landscape,56.476,None
6,15,Peonies and butterfly,6.1899,1906-01-01


## DELETE

## DELETE FROM - Deleting all rows

Running the following command would delete all rows from the table `collections`

In [12]:
conn.execute('''
DELETE FROM "collections";
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


In [13]:
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
SELECT "title", "accession_number", "acquired"
FROM "temp";
""")
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Spring outing,14.769,1914-01-08
3,4,Red landscape,56.476,None
4,5,Peonies and butterfly,6.1899,1906-01-01


## DELETE FROM ___ + WHERE ___ - Deleting selected rows

We can also delete rows that match specific conditions.

For example, to delete the painting “Spring outing” from our table collections we can run:

In [14]:
conn.execute('''
DELETE FROM "collections"
WHERE "title" = 'Spring outing';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,4,Red landscape,56.476,None
3,5,Peonies and butterfly,6.1899,1906-01-01


To delete paintings older than 1909, we can run

In [15]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" < '1909-01-01';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,4,Red landscape,56.476,None


## DELETE FROM ___ + WHERE ___ IS NULL - Deleting rows with NULL values

In [16]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" IS NULL;
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03


## Deleting with Primary and Foreign key Constraints

A foreign key column references the primary key of a different table. If we were to delete the primary key, the foreign key column would have nothing to reference!

Consider now an updated schema for the MFA database, containing information about artwork and artists.The ``artists`` and ``collections`` tables have primary keys—the ID columns. The ``created`` table is the junction table that references these IDs in its two foreign key columns

In [17]:
# Let's add additional tables to raise a foreign key constraint error
mfa_db_add_tables_with_foreign_keys="""

INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Imaginative landscape', '56.496', NULL),
('Spring outing', '14.76', '1914-01-08');

CREATE TABLE "artists" (
    "id" INTEGER,
    "name" TEXT NOT NULL,
    PRIMARY KEY("id")
);

INSERT INTO "artists" ("name")
VALUES
('Li Yin'),
('Qian Weicheng'),
('Unidentified artist'),
('Zhou Chen');

CREATE TABLE "created" (
    "artist_id" INTEGER,
    "collection_id" INTEGER,
    PRIMARY KEY("artist_id", "collection_id"),
    FOREIGN KEY("artist_id") REFERENCES "artists"("id"),
    FOREIGN KEY("collection_id") REFERENCES "collections"("id")
);

INSERT INTO "created" ("artist_id", "collection_id")
VALUES
((SELECT "id" FROM "artists" WHERE "name" = 'Li Yin'), (SELECT "id" FROM "collections" WHERE "title" = 'Imaginative landscape')),
((SELECT "id" FROM "artists" WHERE "name" = 'Qian Weicheng'), (SELECT "id" FROM "collections" WHERE "title" = 'Profusion of flowers')),
((SELECT "id" FROM "artists" WHERE "name" = 'Unidentified artist'), (SELECT "id" FROM "collections" WHERE "title" = 'Farmers working at dawn')),
((SELECT "id" FROM "artists" WHERE "name" = 'Zhou Chen'), (SELECT "id" FROM "collections" WHERE "title" = 'Spring outing'));
"""

conn.executescript(mfa_db_add_tables_with_foreign_keys)
conn.commit()

# Enable foreign key constraints (disabled by default in SQLite)
conn.execute("PRAGMA foreign_keys = ON;")

# Check whether foreign keys are enabled (1 = ON, 0 = OFF)
pd.read_sql_query(
"""
PRAGMA foreign_keys;
""",
conn
)


,foreign_keys
0,1


In [18]:
df=pd.read_sql_query(
    """
    SELECT *
    FROM sqlite_master
    WHERE "type"='table';
    """,
    conn
)
df

,type,name,tbl_name,rootpage,sql
0,table,collections,collections,2,"CREATE TABLE ""collections"" (\n ""id"" INTEGER..."
1,table,temp,temp,4,"CREATE TABLE ""temp"" (\n""title"" TEXT,\n ""acces..."
2,table,artists,artists,5,"CREATE TABLE ""artists"" (\n ""id"" INTEGER,\n ..."
3,table,created,created,6,"CREATE TABLE ""created"" (\n ""artist_id"" INTE..."


Deleting the 'Unidentified artist' (id = 3) from the ``"artists"`` table while it is still referenced in the ``"created"`` table will raise an error:

In [19]:
try:
    conn.execute("""
    DELETE FROM "artists"
    WHERE "name" = 'Unidentified artist';
    """
    )
    conn.commit()
except Exception as e:
    print("Last SQL statement failed. The error was:")
    print(type(e).__name__,":", e)



Last SQL statement failed. The error was:
IntegrityError : FOREIGN KEY constraint failed


## ON DELETE + RESTRICT/NO ACTION/SET NULL/SET DEFAULT/CASCADE

We can specify the action to be taken when an ID referenced by a foreign key is deleted.

To do this, we use the keyword ``ON DELETE`` followed by the action to be taken.

- ``ON DELETE RESTRICT``: This restricts us from deleting IDs

- ``ON DELETE NO ACTION``: This allows the deletion of IDs that are referenced by a foreign key and nothing happens.

- ``ON DELETE SET NULL``: This allows the deletion of IDs that are referenced by a foreign key and sets the foreign key references to NULL.

- ``ON DELETE SET DEFAULT``: This does the same as the previous, but allows us to set a default value instead of NULL.

- ``ON DELETE CASCADE``: This allows the deletion of IDs that are referenced by a foreign key and also proceeds to cascadingly delete the referencing foreign key rows. For example, if we used this to delete an artist or a painting, all the rows linked to the artist or the painting would also be deleted from the ``"created"`` table.

We must add these SQL keywords at the end of the foreign keyword constrain in the table definition and set a new default value for "artist_id":

In [20]:
# Execute multiple SQL statements in a single transaction
conn.executescript(
"""
-- Disable foreign key checks temporarily
PRAGMA foreign_keys = OFF;

-- Create a new table with ON DELETE SET DEFAULT and DEFAULT -1
CREATE TABLE "created_new" (
    "artist_id" INTEGER DEFAULT -1,
    "collection_id" INTEGER,
    PRIMARY KEY ("artist_id", "collection_id"),
    FOREIGN KEY ("artist_id") REFERENCES "artists"("id") ON DELETE SET DEFAULT,
    FOREIGN KEY ("collection_id") REFERENCES "collections"("id") ON DELETE CASCADE
);

-- Add new primary key -1
INSERT INTO "artists" ("id", "name")
VALUES (-1, 'deleted artist');

-- Copy data from the old table
INSERT INTO "created_new"
SELECT * FROM "created";

-- Drop the old table
DROP TABLE "created";

-- Rename the new table
ALTER TABLE "created_new" RENAME TO "created";

-- Re-enable foreign key checks
PRAGMA foreign_keys = ON;
"""
)
# List ON DELETE defined on the "created" table
pd.read_sql_query(
"""
PRAGMA foreign_key_list("created");
""",
conn
)


,id,seq,table,from,to,on_update,on_delete,match
0,0,0,collections,collection_id,id,NO ACTION,CASCADE,NONE
1,1,0,artists,artist_id,id,NO ACTION,SET DEFAULT,NONE


In [21]:
conn.execute("""
DELETE FROM "artists"
WHERE "name" = 'Unidentified artist';
"""
)
conn.commit()
artists=pd.read_sql_query(
    """
    SELECT *
    FROM "artists"
    """,
    conn
)
artists

,id,name
0,-1,deleted artist
1,1,Li Yin
2,2,Qian Weicheng
3,4,Zhou Chen


'Unidentified artist' (id = 3) was succesfully deleted from the ``"artists"`` table while it is still referenced in the ``"created"`` table with -1 :

In [22]:
created=pd.read_sql_query(
    """
    SELECT *
    FROM "created"
    """,
    conn
)
created

,artist_id,collection_id
0,1,3
1,2,1
2,-1,2
3,4,4


## UPDATE ___ SET ___ = ___ WHERE ___

Here is the syntax of the update command:

> UPDATE table
> SET column0 = value0, column1 = value1, ...
> WHERE condition;

Example 1:
Let's change the title of 'Profusion of flowers' painting to 'Flowers':

In [23]:
conn.execute('''
UPDATE "collections"
SET "title"='Flowers',"accession_number"='5.257'
WHERE "title"='Profusion of flowers';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM "collections";', conn)

,id,title,accession_number,acquired
0,1,Flowers,5.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Imaginative landscape,56.496,None
3,4,Spring outing,14.76,1914-01-08


Example 2: A worker find out that the painting “Farmers working at dawn” originally mapped to a “deleted artist” was actually created by the artist Li Yin. Let's update the ``"created"`` table:

In [24]:
conn.execute('''
UPDATE "created"
SET "artist_id" = (
    SELECT "id"
    FROM "artists"
    WHERE "name" = 'Li Yin'
)
WHERE "collection_id" = (
    SELECT "id"
    FROM "collections"
    WHERE "title" = 'Farmers working at dawn'
);
''')
conn.commit()
pd.read_sql_query('SELECT * FROM "created";', conn)

,artist_id,collection_id
0,1,3
1,2,1
2,1,2
3,4,4


## Triggers

A trigger is a SQL statement that runs automatically in response to another SQL statement, such as an INSERT, UPDATE, or DELETE.

Triggers are useful for maintaining data consistency and automating tasks across related tables.

Let's create a new `transactions` table.

In [25]:
conn.execute("""
CREATE TABLE "transactions" (
    "id" INTEGER,
    "title" TEXT,
    "action" TEXT,
    PRIMARY KEY("id")
);
""")

# Commit changes to save the table in the database
conn.commit()
# Let's inspect the schema of the transactions table
pd.read_sql_query(
    "PRAGMA table_info(transactions);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,title,TEXT,0,None,0
2,2,action,TEXT,0,None,0


## CREATE TRIGGER ___ BEFORE DELETE/UPDATE ON___ BEGIN ___ END;  + OLD

IMPORTANT: `OLD` is a special keyword that refers to the row being deleted or updated. If this row had a title column, `OLD."title"` would access that column’s value.

When artwork is sold and deleted from `collections`, we want it automatically logged in `transactions` with an action of “sold”.

In [26]:
# Create the trigger
conn.execute('''
CREATE TRIGGER "sell"
BEFORE DELETE ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (OLD."title", 'sold');
END;
''')

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

,type,name,tbl_name,rootpage,sql
0,trigger,sell,collections,0,"CREATE TRIGGER ""sell""\nBEFORE DELETE ON ""colle..."


In [27]:
pd.read_sql_query('SELECT * FROM "collections";', conn)

,id,title,accession_number,acquired
0,1,Flowers,5.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Imaginative landscape,56.496,None
3,4,Spring outing,14.76,1914-01-08


Let’s test the trigger:

In [28]:
# Execute the DELETE statement
conn.execute("""
DELETE FROM "collections"
WHERE "title" = 'Imaginative landscape';
""")
# Save changes to the database
conn.commit()

In [29]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

,id,title,action
0,1,Imaginative landscape,sold


## CREATE TRIGGER ___ AFTER INSERT/UPDATE ON ___ + NEW

IMPORTANT: `NEW` is a special keyword that refers to the row being inserted or updated. If this row has a title column, `NEW."title"` would accesses that column’s value.


When artwork is bought (inserted into collections), we want it logged in transactions with an action of “bought”.

In [30]:
# Create the trigger
conn.execute("""
CREATE TRIGGER "buy"
AFTER INSERT ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (NEW."title", 'bought');
END;
""")

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

,type,name,tbl_name,rootpage,sql
0,trigger,sell,collections,0,"CREATE TRIGGER ""sell""\nBEFORE DELETE ON ""colle..."
1,trigger,buy,collections,0,"CREATE TRIGGER ""buy""\nAFTER INSERT ON ""collect..."


Let’s test the trigger:

In [31]:
# Execute the INSERT statement
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Flowers', '56.493', NULL);
""")
# Save changes to the database
conn.commit()

In [32]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

,id,title,action
0,1,Imaginative landscape,sold
1,2,Flowers,bought


## Soft Deletions

Soft deletion (or a soft delete) means marking data as deleted rather than actually removing it from the database.


For example, we could add a `deleted` column to the `collections` table with a default value of 0. To “delete” a row, we would update the deleted column to 1. This way, data can be recovered if needed and maintains a complete historical record.

In [33]:
# Add "deleted" column
conn.execute('''
ALTER TABLE "collections"
ADD COLUMN "deleted" INTEGER DEFAULT 0;
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired,deleted
0,1,Flowers,5.257,1956-04-12,0
1,2,Farmers working at dawn,11.6152,1911-08-03,0
2,4,Spring outing,14.76,1914-01-08,0
3,5,Flowers,56.493,None,0


To “delete” a row, we would update the deleted column to 1:

In [34]:
# Update "deleted" column without deleting row
conn.execute('''
UPDATE "collections"
SET "deleted"=1
WHERE "title"='Farmers working at dawn';
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired,deleted
0,1,Flowers,5.257,1956-04-12,0
1,2,Farmers working at dawn,11.6152,1911-08-03,1
2,4,Spring outing,14.76,1914-01-08,0
3,5,Flowers,56.493,None,0


Then, to query only non-deleted rows:

In [35]:
pd.read_sql_query("""
SELECT * FROM "collections"
WHERE "deleted" != 1;
""", conn)

,id,title,accession_number,acquired,deleted
0,1,Flowers,5.257,1956-04-12,0
1,4,Spring outing,14.76,1914-01-08,0
2,5,Flowers,56.493,None,0


This way, data can be recovered if needed and maintains a complete historical record.

In [36]:
conn.close()